### Imports and Constants

In [1]:
# Uncomment, then use ctrl-shift-p and "Developer:Reload Window" if syntax checker does not recognize any of your imports
#%pip install opencv-python numpy tensorflow scikit-learn face-recognition pillow

In [2]:

import cv2,os
import numpy as np
from tkinter import ttk
import tkinter as tk
from tkinter import messagebox
from cv2.typing import MatLike
from my_net_utils import *
from socket import *
from io import BytesIO
import signals, constants
import struct
import face_recognition

from PIL import Image, ImageTk



In [3]:
class App:
    
    def __init__(self) -> None:
        self.make_connection()
        self.get_cam()

        if not self.video_capture.isOpened():
            messagebox.showerror('No Camera Detected','Could not start application due to disconnected camera')
            return
        
        self.take_photo_button = None # :/ unhappy to do this
        self.face_locations = None

        self.root = tk.Tk()
        self.root.title('Start Page')
        try:
            
            self.connection.connect(('192.168.100.11',PORT))
            # self.connection.connect(('localhost',PORT)) 
                #for debugging only
        except:
            try:
                self.make_connection()
                self.connection.connect(('176.29.117.97',PORT))
            except:
                try:
                    self.make_connection()
                    self.connection.connect(('localhost', PORT))
                except Exception as error:
                    self.connection_failure_routine(error)
                    self.quit_app()

        print('Connected')
        self.root.protocol("WM_DELETE_WINDOW",self.quit_app)

        self.root_label = tk.Label(self.root)
        self.video_feed = True

        self.root_label.pack()


        self.take_photo_button = tk.Button(self.root, text="Take Photo", command=self.take_photo)
        self.take_photo_button.pack()
        self.exit_button = tk.Button(self.root, text="Exit", command = self.quit_app) 
        self.exit_button.pack() 

        #canvas.create_window(800,800,window=self.root_label)
    def make_connection(self):
        self.connection = socket()
        print('Socket Created')
        self.connection.settimeout(CL_TIMEOUT)
        print('Timeout set')
        self.connection.setsockopt(SOL_SOCKET,SO_KEEPALIVE,1)
        self.connection.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)

    def get_cam(self):
        self.video_capture = cv2.VideoCapture(0)
        self.video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT,constants.IMG_HEIGHT)
        self.video_capture.set(cv2.CAP_PROP_FRAME_WIDTH,constants.IMG_WIDTH)

    def show_root(self) -> None:
        self.video_feed = True
        if self.take_photo_button and not self.take_photo_button.winfo_viewable():
            self.take_photo_button.pack()
            self.exit_button.pack()
        if not self.video_capture.isOpened():
            self.get_cam()
        
        self.prompt_user_window.destroy()
        try:
            self.connection.sendall(b'')
        except (ConnectionError,TimeoutError) as failure_to_communicate:
            self.connection_failure_routine(failure_to_communicate)

        self.root.deiconify()
        self.update_frame()
        

    
    def quit_app(self):
        self.video_capture.release()
        try:
            self.connection.sendall(signals.QUIT.encode())
            self.connection.shutdown(SHUT_RDWR)
            self.connection.close()
        except ConnectionError:
            return
        finally:
            self.root.destroy()

    

    def launch(self)  -> None:
        self.update_frame()
        self.root.lift()
        self.root.mainloop()  # Start the tkinter main loop

    def get_frame(self)  -> None:
        self.ret, self.last_frame_BGR = self.video_capture.read()  
        if not self.ret:
            return
        self.last_frame_RGB = cv2.cvtColor(self.last_frame_BGR, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        self.showable_last_frame = Image.fromarray(self.last_frame_RGB)
        self.face_locations = face_recognition.face_locations(self.last_frame_BGR)

        return
    
    def take_photo(self):
        if self.exit_button:
            self.exit_button.pack_forget()
        self.video_feed = False
        self.get_frame()
        self.video_capture.release()
        self.prompt_window()



    def login(self)  -> None:
        self.login_button.pack_forget()
        try:
            self.send_image()
            name_in = receive_strip(self.connection)

            if name_in == constants.UNKNOWN:
                messagebox.showinfo('Unknown Student','Last student was not recognized. Please try again.')
                return
            elif name_in == constants.NONE:
                messagebox.showinfo('No face detected','No face could be found in submitted photo. Please try again.')
                return
            

            name = name_in.replace('_', ' ')
            print('Name = ' + name)
            
            result = messagebox.askyesno('Mark as Present?',f'Detected {name}. Is this correct?')

            if result:
                padded_send(self.connection,signals.YES)
            else:
                padded_send(self.connection,signals.NO)
                return

            response_from_server = receive_strip(self.connection)

            messagebox.showinfo('Response from server',f'Server response: {response_from_server}')
            
        except (ConnectionError,TimeoutError) as failure_to_communicate:
            self.connection_failure_routine(failure_to_communicate)
            self.quit_app()
        

    def connection_failure_routine(self,exception):
            error_msg = f'\nError Code: {exception}'
            if isinstance(exception,ConnectionError):
                messagebox.showerror(title='Connection Dropped',
                                     message='Server unexpectedly dropped connection.'
                                     'Please relaunch.\n\n'
                                     f'{error_msg}')
            elif isinstance(exception,TimeoutError):
                messagebox.showerror(title="Connection Timed Out",
                                     message='Connection with server timed out. '
                                     'Please relaunch.\n\n'
                                     f'{error_msg}')
            else:
                messagebox.showerror(title='Unexpected Error',
                                     message='An unexpected error, likely client side, has occurred.'
                                              f'{error_msg}')
                
            
            self.quit_app()
    
    def send_image(self):
        png_buffer = BytesIO()
        self.showable_last_frame.save(png_buffer,format='png')
        start_of_buffer = 0
        png_buffer.seek(start_of_buffer)
        image_size = png_buffer.getbuffer().nbytes
        file_size_bytes = struct.pack('!I',image_size) #Network, unsigned 4 byte int as size is never negative
        self.connection.sendall(file_size_bytes)

        if receive_strip(self.connection) == signals.SIZE_RECEIVED:
            with png_buffer as photo:
                self.connection.sendfile(photo)
        


    def prompt_window(self): 
        try:
            self.connection.sendall(b'')
        except Exception as exc:
            self.connection_failure_routine(exc)
            self.quit_app()
        

        self.root.withdraw() #Hide main window

        img_tk = ImageTk.PhotoImage(image=self.showable_last_frame)  # Convert PIL Image to ImageTk

        #Create window and add widgets
        self.prompt_user_window = tk.Toplevel(self.root)
        self.prompt_user_window.title("What to do with image?")
        self.prompt_user_window.protocol('WM_DELETE_WINDOW',lambda: None) 

        self.prompt_label = tk.Label(self.prompt_user_window)
        self.prompt_label.pack()
        self.prompt_label.image = img_tk
        
        self.prompt_label.config(image=img_tk)  # Update the ImageTk label

        self.login_button = tk.Button(self.prompt_user_window,text='Log in',command=self.login)
        self.login_button.pack()

        self.retake_return_button = tk.Button(self.prompt_user_window,text='Retake Photo/Take New Photo',command=self.show_root)
        self.retake_return_button.pack()

    def update_frame(self):
        if self.video_feed is False:
            return
    
        self.get_frame()  
        if self.face_locations:
            self.face_locations = None
            if self.take_photo_button:
                self.take_photo_button.pack_forget()
                self.root_label.after(1000,self.take_photo)
                self.video_feed = False

        #Block copied from online
        if self.ret:
            img_tk = ImageTk.PhotoImage(image=self.showable_last_frame)  # Convert PIL Image to ImageTk to be able to show
            self.root_label.config(image=img_tk)  # Update the ImageTk label to reflect new image
            self.root_label.image = img_tk  # Keep a reference to prevent garbage collection (ref_count > 0)
            self.root_label.after(30, self.update_frame)  #  ~30fps
        return
    

In [4]:
try:
    instance = App()
    instance.launch()
            
except TimeoutError:
    print('Disconnected')
    
except OSError as e:
    print(f'Failure in sending {e}')

    
print('Program shut')


Socket Created
Timeout set
Socket Created
Timeout set
Socket Created
Timeout set
Connected
Receiving...
APPENDED:SIZE_RECEIVED
RECEIVED SIZE_RECEIVED
Receiving...
APPENDED:20180596 -- Yazan Matarweh
RECEIVED 20180596 -- Yazan Matarweh
Name = 20180596 -- Yazan Matarweh
Sent:  b'YES#############################################################################################################################'
Receiving...
APPENDED:No class is currently ongoing in this room. Please try again later.
RECEIVED No class is currently ongoing in this room. Please try again later.


In [ ]:
#!jupyter nbconvert --to script SendDemo.ipynb